In [1]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer

## Phase1: Clean the initial dataset

In [2]:
df=pd.read_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/GitHub/data/Data/tabular_data/sgCarMart_features+2dates.csv', usecols=lambda x: x!= 'Unnamed: 0')

# drop null
df.dropna(axis=0, inplace=True)

# drop irrelevant columns
data = df.drop(['features', 'accessories', 'descriptions', 'category', 'number_of_owner'], axis=1)

# transform data: extract numerical value from text
# for columns: road_tax, price, depreciation, deregistration value, omv, coe, arf, mileage, engine_cap, power, curb_weight
data.road_tax = df.road_tax.apply(lambda x : 0 if (x == 'N.A' or x == 'N.A.') else re.search('(?<=\$).*(?=\/)', x)[0].replace(',', ''))
data.price = data.price.apply(lambda x : 0 if (x == 'N.A.' or x == 'N.A') else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.depreciation = data.depreciation.apply(lambda x : 0 if (x == 'N.A.' or x == 'N.A') else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.deregistration_value = data.deregistration_value.apply(lambda x : 0 if (x == 'N.A.' or x == 'N.A') else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.omv = data.omv.apply(lambda x : 0 if (x == 'N.A.' or x == 'N.A') else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.coe = data.coe.apply(lambda x : 0 if (x == 'N.A.' or x == 'N.A') else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.arf = data.arf.apply(lambda x : 0 if (re.search('^\$', x) == None) else re.search('(?<=\$).*', x)[0].replace(',', '').strip())
data.mileage = data.mileage.apply(lambda x : re.search('.*(?=km)', x)[0].replace(',', '').strip() if (x.find('km') >= 0 and x != 'N.A.' and x != 'N.A') else 0)
data.engine_cap = data.engine_cap.apply(lambda x: re.search('.*(?=( cc))', x)[0].replace(',', '').strip() if (re.search('( cc)$', x) != None and x != 'N.A' and x != 'N.A.') else 0)
data.power = data.power.apply(lambda x : re.search('.*(?=kW)', x)[0].replace(',', '').strip() if x.find('kW') >= 0 else 0)
data.curb_weight = data.curb_weight.apply(lambda x : re.search('.*(?=kg)', x)[0].replace(',', '').strip() if x.find('kg') >= 0 else 0)

# clean status column to only contain AVAIL and SOLD
data.status = data.status.apply(lambda x: 'AVAIL' if x.upper() != 'SOLD' else 'SOLD')

# correct the datetime columns
data.registration_date = pd.to_datetime(data.registration_date, format='%d-%b-%Y')
data.posted_date = pd.to_datetime(data.posted_date, format='%d-%b-%Y')
data.updated_date = pd.to_datetime(data.updated_date, format='%d-%b-%Y')

# correct the numerical columns
cols = ['manufactured_year', 'road_tax', 'omv', 'coe', 'arf', 'engine_cap', 'power', 'curb_weight']
for c in cols:
    print(c)
    data[c] = pd.to_numeric(data[c])

# export the dataset from phase1 of cleaning
data.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/data/TableauData.csv', index=False)

# peek into the cleaned dataframe
data.head()


manufactured_year
road_tax
omv
coe
arf
engine_cap
power
curb_weight


,model,price,depreciation,registration_date,manufactured_year,mileage,road_tax,transmission,deregistration_value,omv,coe,arf,engine_cap,power,curb_weight,types,status,posted_date,updated_date
1,Audi S4 3.0A TFSI Quattro S-tronic (COE till 0...,108800,17060,2009-10-06,2009,166000,3332,Auto,25472,62346,39936,62346,2995,245.0,1685,Sports Car,AVAIL,2023-04-15,2023-04-15
2,Subaru Forester 2.0i-L Sunroof,0,0,2017-04-28,2017,0,1208,Auto,0,14556,50621,14556,1995,110.0,1591,SUV,SOLD,2023-03-24,2023-04-16
4,Toyota Mark X 2.5A (COE till 05/2026),44000,14090,2006-06-08,2006,172000,2697,Auto,14369,29495,46048,32445,2499,158.0,1500,Luxury Sedan,AVAIL,2023-04-04,2023-04-15
5,Toyota Hiace 3.0M,85800,11050,2021-01-19,2020,0,0,Manual,19602,28235,25255,1412,2982,0.0,1700,Van,AVAIL,2023-04-13,2023-04-13
6,Kia Cerato 1.6A SX,89800,14430,2018-11-19,2018,21725,738,Auto,29552,18186,28457,18186,1591,93.8,1287,Mid-Sized Sedan,AVAIL,2023-04-07,2023-04-15


## Phase 2: Impute the missing values

In [3]:
# import cleaned data
df = pd.read_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/data/TableauData.csv', usecols=lambda x : x!= 'Unnamed: 0')

# add maker column to reduce the dimension of model
print('total data: ', df.shape[0])
print('unique model: ', len(df.model.unique()))
df['maker'] = df.model.apply(lambda x : x.split(' ')[0].upper())
# standardize the maker tag
df.loc[df['maker']=='LAND', 'maker'] = 'LAND-ROVER'
df.loc[df['maker'] == 'MERCEDES-BENZ', 'maker'] = 'MERCEDES'
print('unique maker: ', len(df.maker.unique()))

# impute missing value for numerical columns
cols = ['price', 'depreciation', 'manufactured_year', 'mileage', 'road_tax', 'deregistration_value', 'omv', 'coe', 'arf', 'engine_cap', 'power', 'curb_weight']
# replace all 0 to null
for c in cols:
    df[c] = df[c].apply(lambda x : np.NaN if x == 0 else x)
# select only numerical columns
toImpute = df[cols]
# use KNNImputer to impute the missing values
imputer = KNNImputer(n_neighbors=5, weights="distance")
imputedData = pd.DataFrame(imputer.fit_transform(toImpute), columns=cols)
# round up the imputed value
import math
for c in cols:
    imputedData[c] = imputedData[c].apply(lambda x : math.ceil(x))
# replace missing values inside the initial dataset with imputed data    
df[cols] = imputedData

# export dataframe to csv file
df.to_csv('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/data/TableauData.csv', index=False)

# peek into the final dataframe
df.head()


total data:  4790
unique model:  2434
unique maker:  59


,model,price,depreciation,registration_date,manufactured_year,mileage,road_tax,transmission,deregistration_value,omv,coe,arf,engine_cap,power,curb_weight,types,status,posted_date,updated_date,maker
0,Audi S4 3.0A TFSI Quattro S-tronic (COE till 0...,108800,17060,2009-10-06,2009,166000,3332,Auto,25472,62346,39936,62346,2995,245,1685,Sports Car,AVAIL,2023-04-15,2023-04-15,AUDI
1,Subaru Forester 2.0i-L Sunroof,63368,15194,2017-04-28,2017,93852,1208,Auto,26761,14556,50621,14556,1995,110,1591,SUV,SOLD,2023-03-24,2023-04-16,SUBARU
2,Toyota Mark X 2.5A (COE till 05/2026),44000,14090,2006-06-08,2006,172000,2697,Auto,14369,29495,46048,32445,2499,158,1500,Luxury Sedan,AVAIL,2023-04-04,2023-04-15,TOYOTA
3,Toyota Hiace 3.0M,85800,11050,2021-01-19,2020,43313,453,Manual,19602,28235,25255,1412,2982,92,1700,Van,AVAIL,2023-04-13,2023-04-13,TOYOTA
4,Kia Cerato 1.6A SX,89800,14430,2018-11-19,2018,21725,738,Auto,29552,18186,28457,18186,1591,94,1287,Mid-Sized Sedan,AVAIL,2023-04-07,2023-04-15,KIA


## convert the LTA table to machine readable

In [4]:
import pandas as pd

df_lta = pd.read_excel('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/data/LTA_Annual_Transfer_Ownership.xlsm')
df_lta.head()

,Maker,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ALFA,173,142,133,178,164,125,92,82,79,98,56
1,ALPINA,0,0,0,1,3,8,6,12,11,6,14
2,ALPINE,0,0,1,0,0,0,1,1,3,6,5
3,ASTON,30,40,29,36,37,35,30,41,54,49,44
4,AUDI,1935,1622,1819,2293,2659,3106,3091,2914,2999,3518,3221


In [5]:
# melt the table
df_melted = pd.melt(df_lta, id_vars='Maker', var_name='Year', value_name='Sales')
df_melted.head()

,Maker,Year,Sales
0,ALFA,2012,173
1,ALPINA,2012,0
2,ALPINE,2012,0
3,ASTON,2012,30
4,AUDI,2012,1935


In [6]:
df_melted.to_excel('/Users/eesoonhang/Library/CloudStorage/OneDrive-Personal/04. MTech_Y2_Sem1/4.0 Project/data/LTA_Annual_Transfer_Ownership_melted.xlsx')